#### VER 1.7 (Last7 DAYS, Merge 7 days, Filter, EQ , Sat and Sunday excluded)

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to download bhav copy data
def download_bhav_copy(url, save_path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded and saved to {save_path}")
            return True
        else:
            print(f"Failed to download from {url}, status code: {response.status_code}")
            return False
    except requests.RequestException as e:
        print(f"Error during download: {e}")
        return False

# Initialize variables
base_url = "https://archives.nseindia.com/products/content/sec_bhavdata_full_"
save_dir = "D:\\[01] Python Projects For Trading\\bhav_copy\\"
os.makedirs(save_dir, exist_ok=True)

# Date variables
days_fetched = 0
days_to_fetch = 7
current_date = datetime.now()

# Download data for the last 7 trading days excluding weekends
file_paths = []
while days_fetched < days_to_fetch:
    # Check if the current day is a weekend
    if current_date.weekday() < 5:  # weekdays: Monday=0, ..., Friday=4
        formatted_date = current_date.strftime("%d%m%Y")
        file_name = f"sec_bhavdata_full_{formatted_date}.csv"
        save_path = os.path.join(save_dir, file_name)
        url = f"{base_url}{formatted_date}.csv"
        
        if download_bhav_copy(url, save_path):
            file_paths.append(save_path)
            days_fetched += 1
    
    # Move to the previous day
    current_date -= timedelta(1)

# Create a master file by combining all data
master_file_path = os.path.join(save_dir, "master_bhav_copy.csv")
all_dataframes = [pd.read_csv(file) for file in file_paths]
master_df = pd.concat(all_dataframes, ignore_index=True)
master_df.to_csv(master_file_path, index=False)
print(f"Master file created at {master_file_path}")

# Filter the master file to create EQ_file
eq_file_path = os.path.join(save_dir, "EQ_file.csv")
eq_df = master_df[master_df[" SERIES"] == " EQ"]
eq_df.to_csv(eq_file_path, index=False)
print(f"EQ file created at {eq_file_path}")


### Transpose the data for easy view

In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r"D:\[01] Python Projects For Trading\bhav_copy\EQ_file.csv")

# Pivot the DataFrame
df_pivot = df.pivot(index='SYMBOL', columns=' DATE1', values=' DELIV_PER')

# Flatten the column names
df_pivot.columns = [f'{col} DELIV_PER' for col in df_pivot.columns]

# Reset index to include 'SYMBOL' as a column
df_pivot = df_pivot.reset_index()

# Save the transformed DataFrame to a new CSV file
df_pivot.to_csv('transformed_file.csv', index=False)

print("Transformation complete. The new file is saved as 'transformed_file.csv'.")


Transformation complete. The new file is saved as 'transformed_file.csv'.
